In [5]:
import requests
from datetime import datetime
import pandas as pd
import json

response = requests.get("https://ogd-static.voteinfo-app.ch/v1/ogd/sd-t-17-02-20241124-eidgAbstimmung.json")

bfs_data = response.json()
print("Data successfully loaded")

with open('voting_data.json', 'w', encoding='utf-8') as f:
   json.dump(bfs_data, f, ensure_ascii=False, indent=2)

def get_short_title(title):
   import re
   match = re.search(r'\((.*?)\)', title)
   return match.group(1) if match else title

vorlagen = [{
   'id': vorlage['vorlagenId'],
   'title_de': get_short_title(next(t['text'] for t in vorlage['vorlagenTitel'] if t['langKey'] == 'de')),
   'title_fr': get_short_title(next(t['text'] for t in vorlage['vorlagenTitel'] if t['langKey'] == 'fr'))
} for vorlage in bfs_data['schweiz']['vorlagen']]

results_df = pd.DataFrame(columns=['id', 'time', 'percent_yes', 'percent_no', 'lower_bound', 'upper_bound', 'share_counted', 'label', 'result', 'title_de', 'title_fr', 'type', 'standesstimmen'])

for i, vorlage in enumerate(vorlagen):
   timestamp = int(datetime.now().timestamp() * 1000)
   url = f"https://tamedia-abstimmungszentrale-hochrechnung.storage.googleapis.com/{vorlage['id']}.csv?{timestamp}"
   
   try:
       leewas_data = pd.read_csv(url)
       latest_row = leewas_data.sort_values('Time').iloc[-1]
       
       percent_yes = latest_row['Estimate']
       result_type = 'trend'
       
       if bfs_data['schweiz']['vorlagen'][i]['resultat']['gebietAusgezaehlt']:
           percent_yes = bfs_data['schweiz']['vorlagen'][i]['resultat']['jaStimmenInProzent']
           result_type = 'final'
           
       new_row = pd.DataFrame({
            'id': [vorlage['id']],
            'time': [latest_row['Time']], 
            'percent_yes': [round(percent_yes, 2)],
            'percent_no': [round(100 - percent_yes, 2)],
            'lower_bound': [latest_row['lower_bound']],
            'upper_bound': [latest_row['upper_bound']],
            'share_counted': [latest_row['Share_counted']],
            'label': [latest_row['label']],
            'result': [latest_row['result']],
            'title_de': [vorlage['title_de']],
            'title_fr': [vorlage['title_fr']],
            'type': [result_type],
            'standesstimmen': [latest_row['Standesstimmen']] 
        })
   except:
       new_row = pd.DataFrame({
            'id': [vorlage['id']],
            'time': [timestamp],
            'percent_yes': [None],
            'percent_no': [None],
            'lower_bound': [None],
            'upper_bound': [None],
            'share_counted': [None],
            'label': [None],
            'result': [None],
            'title_de': [vorlage['title_de']],
            'title_fr': [vorlage['title_fr']],
            'type': ['trend'],
            'standesstimmen': [None]
        })
   
   results_df = pd.concat([results_df, new_row], ignore_index=True)

results_df.to_csv("leewas_data.csv", index=False)

Data successfully loaded


/var/folders/c8/bvk5bjs15mzg5h8m_jcbywx40000gn/T/ipykernel_1200/832727024.py:74: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_row], ignore_index=True)


,id,time,percent_yes,percent_no,lower_bound,upper_bound,share_counted,label,result,title_de,title_fr,type,standesstimmen
0,6730,16:49:50,47.30,52.70,46.89,47.69,1.0,Klarer Trend,Nein,Bundesbeschluss über den Ausbauschritt 2023 fü...,Arrêté fédéral sur l’étape d’aménagement 2023 ...,final,-1
1,6740,16:51:10,48.42,51.58,48.03,48.83,1.0,Klarer Trend,Nein,Mietrecht: Untermiete,droit du bail: sous-location,final,-1
2,6750,16:53:57,46.17,53.83,45.78,46.58,1.0,Klarer Trend,Nein,Mietrecht: Kündigung wegen Eigenbedarfs,droit du bail: résiliation pour besoin propre,final,-1
3,6760,16:55:20,53.31,46.69,52.86,53.66,1.0,Klarer Trend,Ja,KVG,LAMal,final,-1
